# 인구 유출입 데이터 시각화

In [1]:
# 시각화를 위한 라이브러리 임포트
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# # csv 호출
csv = pd.read_csv('Resident_Population_Graph.csv')

In [3]:
# 호출한 csv를 데이터프레임으로 변환
df_main = pd.DataFrame(csv)
df_main.head()

,행정동,소계,year,소계_lag,delta
0,가음정동,39332.0,201501,NaN,NaN
1,가음정동,39163.0,201502,39332.0,-169.0
2,가음정동,39076.0,201503,39163.0,-87.0
3,가음정동,39103.0,201504,39076.0,27.0
4,가음정동,39046.0,201505,39103.0,-57.0


In [4]:
# "의창구, 성산구, 마산합포구, 마산회원구, 진해구" 행정동만 추출하여 별개의 데이터프레임으로 저장
target_districts = ["의창구", "성산구", "마산합포구", "마산회원구", "진해구"]
df_filtered = df_main[df_main['행정동'].isin(target_districts)]

In [ ]:
# 행정동 편입에 따른 이상치 ( 의창구 용지동 -> 성산구 용지동 ) 변화에 대응하기위해 편입 당월 데이터의 흐름 무시 21.06~21.07


In [ ]:
# 한글폰트 다운로드
# if u need .'sudo apt-get install fonts-nanum*'
# !apt-get install -y fonts-nanum
# !fc-cache -fv

In [ ]:
import matplotlib.font_manager as fm
font_list = [f.name for f in fm.fontManager.ttflist if 'Nanum' in f.name]

print(font_list)

In [ ]:
# Linux 환경에서 한글 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rc('font', family=fontprop.get_name())

In [ ]:
import matplotlib.ticker as mticker

# 그룹 재생성
grouped = df_main.groupby(['행정동'])
grouped_dfs = [group for _, group in grouped]

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 20))
axes = axes.flatten()

x_min = df_main['year'].min()
x_max = df_main['year'].max()

# y축을 0 기준으로 대칭 설정
max_abs = max(abs(df_main['delta'].min()), abs(df_main['delta'].max()))
y_min, y_max = -max_abs, max_abs
y_center = 0

for ax, group in zip(axes, grouped_dfs):
    g = group
    if g.empty:
        ax.set_axis_off()
        continue
    sns.lineplot(data=g, x='year', y='delta', ax=ax)

    ax.set_title(f"{g['행정동'].values[0]}", fontsize=9)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

    # x축(0선)을 y=0에 위치
    ax.spines['bottom'].set_position(('data', 0))
    ax.spines['top'].set_visible(False)
    ax.tick_params(axis='x', bottom=True, top=False, labelbottom=True, labeltop=False)

    # y축 눈금/그리드 유지
    ax.yaxis.set_major_locator(mticker.MultipleLocator(20000))
    ax.yaxis.set_minor_locator(mticker.AutoMinorLocator(4))
    ax.grid(which='major', axis='y', linestyle='-', color='lightgray', linewidth=0.8)
    ax.grid(which='minor', axis='y', linestyle=':', color='lightgray', alpha=0.5)

    # x축 그리드
    ax.xaxis.set_major_locator(mticker.MaxNLocator(5))
    ax.grid(which='major', axis='x', linestyle='--', color='lightgray', alpha=0.6)

    ax.tick_params(axis='x', labelrotation=45)
    

# 남은 서브플롯 비활성화
for ax in axes[len(grouped_dfs):]:
    ax.set_axis_off()

plt.tight_layout()
plt.show()